In [1]:
# !pip install -q openai
# !pip install -q langchain
# !pip install -q guardrails-ai
# !pip install -q faiss-cpu
# !pip install -q pypdf
# !pip install -q python-dotenv
# !pip install -q datasets
# !pip install -q huggingface_hub

In [1]:
import os
from dotenv import load_dotenv
import json

from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage,  SystemMessage

#Guardrails
import openai
from rich import print
from langchain.output_parsers import GuardrailsOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

## Load Environment Varible

In [2]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")
huggingface_api_key = os.environ.get("HUGGINGFACE_API_KEY")

## Loading the Document

In [3]:
# loader = PyPDFDirectoryLoader("/content/sample_data/Data/")
loader = PyPDFDirectoryLoader("../data")
# loader = PyPDFDirectoryLoader("../cyber")
data = loader.load()

In [4]:
print(data[0].page_content[:100])

© 2020 . D
r. Zhi Gang Sha  & Rulin Xiu . This is a research/review paper, distributed under the ter

In [5]:
file_name = "Extracted_data.txt"

# Open the file in write mode
with open(file_name, "w",encoding="utf-8") as file:
    for item in data:
        # Extract the "page_content" attribute from each item in the list
        page_content = item.page_content

        # Write the page_content to the file followed by a newline
        file.write(page_content + "\n")

print(f"Data has been written to {file_name}")

Data has been written to Extracted_data.txt

Chunking the Documents

In [6]:
#Step 05: Split the Extracted Data into Text Chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=500)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=500)
text_chunks = text_splitter.split_documents(data)
print("Lenth of the whole documentation is:",len(text_chunks))

Lenth of the whole documentation is: 364

## Prompt template

[INPUT TEXT]

[CONTEXT ]

[REQUEST FOR Q+A]

[RESPONSE SAMPLE]


Getting Context of the Data

In [7]:
chat = ChatOpenAI(temperature=0.6) # for sythetic data generation
model = OpenAI(temperature=0) # for parsing the output

In [8]:
messages = [
    HumanMessage(
        content=f"{data[0].page_content}give the above information give me a single line rich summary with keywords in the beginning which can be used to descibe the entiner inforamtion"
    ),
]
response = chat(messages)

print(response.content)

Summary: This research paper proposes a concept called Information Mechanics, which suggests that information 
determines observed phenomena in the universe. The interaction of information space and time creates everything we 
observe. The paper introduces the information action and function, which represent the maximum possibilities and 
calculate the possible states in a system, respectively. The study addresses various problems in physics, such as 
the hierarchy problem, cosmological constant problem, dark matter, dark energy, and black holes, and relates them 
to the concept of "It from Bit." Keywords: information mechanics, hierarchy problem, cosmological constant problem,
dark matter, dark energy, black hole, it from bit, grand unification theory.

## Generating QnA 

- *prompt* : provide {questions_per_chunk} question and answer pairs base on the text above , The Question must begin with \n"In the context of ...\".The answer borrow, verbatim, from the text above. In providing each question consider that the reader does not see of have access to any of the other questions from context. Vary the style and formate fo quesitons. Respond in plain test on a new line for each question and answer. Do not include Do no include qestion numbers, Here is an exmaple of two question and answer paids:\n\n {tain_sample}

In [9]:
dataset_name = "TaoGPT-v1.json"

json_response_format = [
                {
                    "question": "In the context of ...",
                    "answer": "..."
                },
                {
                    "question": "In the context of ...",
                    "answer": "..."
                },
                {
                    "question": "In the context of ...",
                    "answer": "..."
                }
            ]

rail_spec = """
<rail version="0.1">
<output>
<list name="data" description="list of question answer pairs">
    <object>
        <string name="question" description="the question"/>
        <string name="answer"  description="the answer"/>
    </object>
</list>
</output>
<prompt>

Given the following list of json question and answer paids , please extract it in a proper JSON formate 
${generated_data_json}

${gr.complete_json_suffix_v2}
</prompt>
</rail>
"""
def write_to_json_file(json_data, json_file_name):
    try:
        try:
            with open(json_file_name, "r") as outfile:
                data = json.load(outfile)
            for json_data_pairs in json_data:
                data.append(json_data_pairs)
            with open(json_file_name, "w") as outfile:
                json.dump(data, outfile)
        except FileNotFoundError:
            with open(json_file_name, "w") as outfile:
                json.dump(json_data, outfile)        
    except Exception as e:
        print("Error in write_to_json_file", e)

# def write_to_json_file(json_data_list, json_file_name):
#   try:
#     with open(json_file_name, "r") as outfile:
#       data = json.load(outfile)
    
#     for json_data in json_data_list:
#       data.append(json_data)

#     with open(json_file_name, "w") as outfile: 
#       json.dump(data, outfile)

#   except FileNotFoundError:
#     data = json_data_list  
#     with open(json_file_name, "w") as outfile:
#       json.dump(data, outfile)

#   except Exception as e:
#     print("Error in write_to_json_file", e)

In [14]:
for text in text_chunks[:2]:
    try:
        context_messages = [
            HumanMessage(
                content=f"{text.page_content}give the above information give me a single line rich summary with keywords in the beginning which can be used to descibe the entiner inforamtion"
            ),
        ]
        context_response = chat(context_messages)
        print(context_response.content)
        try:
            generated_messages = [
                HumanMessage(
                    content=f"""given the context:{context_response} 
                    and the information {text.page_content} provide 5 question and answer pairs base on the text above , 
                    The Question must begin with 
                    "In the context of ...\".The answer borrow, verbatim, from the text above. In providing each question consider that the reader does not see of have access to any of the other questions from context. Vary the style and formate fo quesitons. 
                    Respond in only JSON following this formate
                    {json_response_format} and nothing else"""
                ),
            ]
            generated_response = chat(generated_messages)
            try:
                eval_generated_json = eval(generated_response.content)
                if len(eval_generated_json) >= 1:
                    write_to_json_file(eval_generated_json, dataset_name)
                    print(eval_generated_json)
                else:
                    raise Exception("Data generated not in the right format")
            except:
                # output_parser = GuardrailsOutputParser.from_rail_string(extractor_rail)
                output_parser = GuardrailsOutputParser.from_rail_string(rail_spec, api=openai.ChatCompletion.create)
                prompt = PromptTemplate(
                    template=output_parser.guard.base_prompt,
                    input_variables=output_parser.guard.prompt.variable_names,
                )

                model = OpenAI(temperature=0) # type: ignore
                generated_response_final = model(prompt.format_prompt(generated_data_json=generated_response.content).to_string())
                generated_response_final = output_parser.parse(generated_response_final)
                generated_response_final = generated_response_final["data"]
                print(generated_response_final)                
                write_to_json_file(generated_response_final, dataset_name)
        except:
            print("Error in generating the data")
    except Exception as e:
        print("Failed to get context of the text")
    print("\n---------------------------------------------------\n")
    


Summary: This research paper proposes Information Mechanics as a framework to understand the process of "It from 
Bit," where information gives rise to physics. The authors suggest that information determines observed phenomena 
and introduce information space and time as the basic elements. They derive an action, the information action, and 
the information function to represent the maximum possibilities of information in a system. This paper provides 
insights into the dynamics of information and its role in shaping our reality.

Keywords: Information Mechanics, It from Bit, observed phenomena, information space, information time, information 
action, information function.

[
    {
        'question': 'In the context of this research paper, what is proposed as a framework to understand the 
process of "It from Bit"?',
        'answer': 'Information Mechanics'
    },
    {
        'question': 'In the context of this research paper, what are the two basic elements introduced to represent
information?',
        'answer': 'information space and time'
    },
    {
        'question': 'In the context of this research paper, what does the information action represent?',
        'answer': 'the maximum possibilities of information in a system'
    },
    {
        'question': 'In the context of this research paper, what does the information function represent?',
        'answer': 'the maximum possibilities of information in a system'
    },
    {
        'question': 'In the context of this research paper, what do the authors suggest determines observed 
phenomena?',
        'answer': 'information'
    }
]

---------------------------------------------------

The interaction of information space and time creates everything we observe, leading to the development of 
information mechanics. This theory addresses various phenomena such as the hierarchy problem, cosmological constant
problem, dark matter, dark energy, black holes, and the concept of "it from bit." The information action and 
information function play crucial roles in understanding the maximum possibilities and possible states within a 
system. Keywords: information mechanics, hierarchy problem, cosmological constant problem, dark matter, dark 
energy, black hole, it from bit, grand unification theory.

[
    {
        'question': 'In the context of information mechanics, what does the information action represent?',
        'answer': 'The information action represents the maximum possibilities, i.e., information, in a system.'
    },
    {
        'question': 'In the context of information mechanics, what does the information function calculate?',
        'answer': 'The information function calculates the possible states in a system.'
    },
    {
        'question': 'In the context of information mechanics, what are some phenomena addressed by this theory?',
        'answer': 'The theory addresses various phenomena such as the hierarchy problem, cosmological constant 
problem, dark matter, dark energy, black holes, and the concept of "it from bit."'
    },
    {
        'question': 'In the context of information mechanics, what are the two basic elements making up 
information?',
        'answer': 'The two basic elements making up information are information space and time.'
    },
    {
        'question': 'In the context of information mechanics, what theory does the information action appear to be 
similar to?',
        'answer': 'The information action appears to be similar to the string action in string theory and 
superstring theory but with a different meaning.'
    }
]

---------------------------------------------------

## HuggingFace

In [15]:
from huggingface_hub import notebook_login
from huggingface_hub import HfApi
from datasets import load_dataset
api = HfApi(token=huggingface_api_key)


In [16]:
dataset = load_dataset('json', data_files="./TaoGPT-v1.json")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/Adithya/.cache/huggingface/datasets/json/default-08b5836d41bc1055/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 20
    })
})

In [17]:
# dataset.push_to_hub("Dataset name")
dataset.push_to_hub("agency888/TaoGPT-v1")

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
# Pushing Dataset.json
api.upload_file(
    path_or_fileobj="./TaoGPT-v1.json",
    path_in_repo="TaoGPT-v1.json",
    repo_id="agency888/TaoGPT-v1",
    repo_type="dataset",
)

'https://huggingface.co/datasets/agency888/TaoGPT-v1/blob/main/TaoGPT-v1.json'

In [28]:
# Pushing Dataset Readme
api.upload_file(
    path_or_fileobj="./README.md",
    path_in_repo="README.md",
    repo_id="agency888/TaoGPT-v1",
    repo_type="dataset",
)

'https://huggingface.co/datasets/agency888/TaoGPT-v1/blob/main/README.md'